In [1]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

In [68]:
rho = 1.225 # masse volumique air (kg/m3)
Cbetz = 16/27 # limite de Betz
d = 1e-2 # diamètre
A = np.pi * d**2 / 4 # aire

# Puissance du vent en entrée
$$
P_{vent} = \dfrac{1}{2} \rho A V^3
$$
# Puissance électrique en sortie
$$
P_{elec} = \eta_{meca/elec} C_p P_{vent} < \eta_{meca/elec} \dfrac{16}{27} P_{vent}
$$

In [114]:
from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider, HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook
output_notebook()

x = np.linspace(0, 5, 11)
y1 = .5 * rho * A * x**3 * 1000
y2 = Cbetz * .5 * rho * A * x**3 * 1000
y3 = (.8 * .1) * .5 * rho * A * x**3 * 1000

source = ColumnDataSource(data=dict(x=x, y1=y1, y2=y2, y3=y3))
constants = dict(rho=rho)

hover = HoverTool(tooltips=[
    ("Pvent", "@y1 mW"),
    ("PBetz", "@y2 mW"),
    ("Pelec", "@y3 mW")
])

plot = figure(y_range=(-5, 10), plot_width=700, plot_height=600, title="Puissance en fonction de la vitesse de vent", tools=[hover])

plot.line('x', 'y1', source=source, line_width=3, line_alpha=0.6, color='green', legend = "Puissance du vent")
plot.line('x', 'y2', source=source, line_width=3, line_alpha=0.6, color='red', legend = "Puissance Betz")
plot.line('x', 'y3', source=source, line_width=3, line_alpha=0.6, color='blue', legend = "Puissance électrique")
plot.legend.orientation = "horizontal"

plot.title.text_font_size = "15pt"
plot.xaxis.axis_label = r"Vitesse du vent (m/s)"
plot.yaxis.axis_label = "Puissance (mW)"
plot.axis.axis_label_text_font_size = "13pt"
#plot.axis.axis_label_text_font_style = "bold"

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var k = diam.value;
    var Cp = Cp.value;
    var rend = rend.value;
    x = data['x']
    y1 = data['y1']
    y2 = data['y2']
    y3 = data['y3']
    for (i = 0; i < x.length; i++) {
        y1[i] = .5 * 1.225 * x[i]**3 * (3.141592 * (k/1000)**2 / 4) * 1000;
        y2[i] = (16/27) * .5 * 1.225 * x[i]**3 * (3.141592 * (k/1000)**2 / 4) * 1000;
        y3[i] = (rend / 100) * (Cp / 100) * .5 * 1.225 * x[i]**3 * (3.141592 * (k/1000)**2 / 4) * 1000; 
    }
    source.change.emit();
""")

diam_slider = Slider(start=10, end=50, value=10, step=1,
                    title="Diamètre du rotor (mm)", callback=callback)
callback.args["diam"] = diam_slider

Cp_slider = Slider(start=1, end=59, value=10, step=1,
                    title="Cp (%)", callback=callback)
callback.args["Cp"] = Cp_slider

rend_slider = Slider(start=10, end=100, value=80, step=1,
                    title="Rendement méca/élec (%)", callback=callback)
callback.args["rend"] = rend_slider

layout = row(
    plot,
    widgetbox(diam_slider, Cp_slider, rend_slider),
)

#output_file("slider.html", title="slider.py example")

Loading BokehJS ...

In [115]:
show(layout)